# P-Multigrid + Jacobi Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

## p=4 -> p=1, v=1

In [ ]:
# setup
finep = 4
coarsep = 1
dimension = 2
numbercomponents = 1
mesh = Mesh2D(1.0, 1.0)

ctofbasis = TensorH1LagrangePProlongationBasis(coarsep+1, finep+1, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep+1, finep+1, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep+1, finep+1, mesh)

# Jacobi smoother
jacobi = Jacobi(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, jacobi, [ctofbasis])

In [ ]:
# full operator symbols
numbersteps = 100
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2
θ_step = 2π/(numbersteps-1)
θ_range = θ_min:θ_step:θ_max
ω_min = 0.0
ω_max = 1.1
ω_step = (ω_max-ω_min)/(numbersteps-1)
ω_range = ω_min:ω_step:ω_max

# compute and plot smoothing factor
# setup
eigenvalues = zeros(numbersteps)

# compute
for i in 1:numbersteps, j in 1:numbersteps
    θ = [θ_range[i], θ_range[j]]
    if sqrt(abs(θ[1])^2 + abs(θ[2])^2) > π/256
        M = computesymbols(multigrid, [1.0], [0, 0], θ)
        S = I - computesymbols(jacobi, [1.0], θ)
        for w in 1:numbersteps
            ω = ω_range[w]
            currenteigenvalues = [abs(val) for val in eigvals((I - ω*S) * M * (I - ω*S))]
            eigenvalues[w] = max(eigenvalues[w], currenteigenvalues...)
        end
    end
end

# plot
xrange = ω_range
plot(
    xrange,
    xlabel="ω",
    xtickfont=font(12, "Courier"),
    eigenvalues,
    ytickfont=font(12, "Courier"),
    ylabel="spectral radius",
    linewidth=3,
    legend=:none,
    title="Two-Grid Convergence Factor",
    palette=palette(:tab10)
)
ylims!(min(0.0, eigenvalues...) * 1.1, max(eigenvalues...) * 1.1)

In [ ]:
savefig("two_grid_converge_4_to_1_1_smooth_2d")

## p=4 -> p=1, v=2

In [ ]:
# setup
finep = 4
coarsep = 1
dimension = 2
numbercomponents = 1
mesh = Mesh2D(1.0, 1.0)

ctofbasis = TensorH1LagrangePProlongationBasis(coarsep+1, finep+1, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep+1, finep+1, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep+1, finep+1, mesh)

# Jacobi smoother
jacobi = Jacobi(finediffusion)

# p-multigrid preconditioner
multigrid = PMultigrid(finediffusion, coarsediffusion, jacobi, [ctofbasis])

In [ ]:
# full operator symbols
numbersteps = 100
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2
θ_step = 2π/(numbersteps-1)
θ_range = θ_min:θ_step:θ_max
ω_min = 0.0
ω_max = 1.1
ω_step = (ω_max-ω_min)/(numbersteps-1)
ω_range = ω_min:ω_step:ω_max

# compute and plot smoothing factor
# setup
eigenvalues = zeros(numbersteps)

# compute
for i in 1:numbersteps, j in 1:numbersteps
    θ = [θ_range[i], θ_range[j]]
    if sqrt(abs(θ[1])^2 + abs(θ[2])^2) > π/256
        M = computesymbols(multigrid, [1.0], [0, 0], θ)
        S = I - computesymbols(jacobi, [1.0], θ)
        for w in 1:numbersteps
            ω = ω_range[w]
            currenteigenvalues = [abs(val) for val in eigvals((I - ω*S)^2 * M * (I - ω*S)^2)]
            eigenvalues[w] = max(eigenvalues[w], currenteigenvalues...)
        end
    end
end

# plot
xrange = ω_range
plot(
    xrange,
    xlabel="ω",
    xtickfont=font(12, "Courier"),
    eigenvalues,
    ytickfont=font(12, "Courier"),
    ylabel="spectral radius",
    linewidth=3,
    legend=:none,
    title="Two-Grid Convergence Factor",
    palette=palette(:tab10)
)
ylims!(min(0.0, eigenvalues...) * 1.1, max(eigenvalues...) * 1.1)

In [ ]:
savefig("two_grid_converge_4_to_1_2_smooth_2d")